# MAPS Step 1 - Image Quality Control
After collecting images, follow the steps below to remove out-of-focus images. Artifacts such as foreign objects, air bubbles and over-expressed cells will be eliminated when measuring focus. <br>
Some outputs were saved to show the expected output.

In [1]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from MAPS_util import *

## Set the working directory

In [2]:
# check current directory
os.getcwd()

'/Users/jessesnaust/Documents/python scripts/MAPS_util'

In [8]:
# change directory, if needed
os.chdir('/volumes/WD_1TB/Cellomics_images/')
# confirm the directory
os.getcwd()

'/Volumes/WD_1TB/Cellomics_images'

## Perform image QC
### Note: This program assumes there are 3 channels per field. <br> Therefore, 1 image set has blue, green and red channels.
For example: blue-DAPI, green-GFP, red-TRITC.
Each channel is saved as separate files with suffixes d0, d1 and d2.

In [32]:
folder_path = "K342N-102319/" # source

create_folder(folder_path, "not-blurry") # create destination folder, if it doesn't already exist
dst_folder = folder_path + "not-blurry/" # destination

Created folder 'not-blurry'.


In [33]:
# get the file names of the images in the folder
ID_list = get_names(folder_path)
print(f'There are {len(ID_list)} set of images waiting to be processed.')

There are 300 set of images waiting to be processed.


In [34]:
for n, x in enumerate(ID_list):
    blue = None
    green = None
    red = None
    print()
    print(f'Processing image set {n+1} out of {len(ID_list)}. ID={x}')

    # load each channel
    for entry in os.scandir(folder_path): # import each channel
        if x in entry.name and entry.name.endswith('d0.TIF'): # change the suffix of your image files to match
            blue = cv2.imread(entry.path, 0)
            print(f"blue: {entry.name}")
        if x in entry.name and entry.name.endswith('d1.TIF'):
            green = cv2.imread(entry.path, 0)
            print(f"green: {entry.name}")
        if x in entry.name and entry.name.endswith('d2.TIF'):
            red = cv2.imread(entry.path, 0)
            print(f"red: {entry.name}")
    # check if all channels are correctly loaded
    if blue is None:
        print("No blue channel found. Skip...")
        continue
    elif green is None:
        print("No green channel found. Skip...")
        continue
    elif red is None:
        print("No red channel found. Skip...")
        continue
    
    # use DAPI (blue channel) to do the 1st rnd of blurry removal
    edgesBlue = cv2.Canny(blue, 100, 200)
    fmBlue = variance_of_laplacian(edgesBlue)
    
    if fmBlue > 500: # focus threshold, higher = sharper
        print(f'Image might be sharp... 1st focus check FM1={fmBlue:.2f}')
        # blur out over-exposed cells
        mask = find_blobs(red)
        blrred = blur_blobs(red, mask)
                
        # remove air bubbles
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))  # taking a matrix of size 5 as kernel
        morph = cv2.morphologyEx(blrred, cv2.MORPH_OPEN, kernel) # (erosion followed by dilation)
            
        # use TRITC (red channel) to do the 2nd rnd of blurry removal
        edgesRed = cv2.Canny(morph, 50, 200)
        fmRed = variance_of_laplacian(edgesRed)
            
        if fmRed > 200: # focus threshold, higher = sharper
            print(f'Image is sharp! 2nd focus check FM2={fmRed:.2f}')
            # uncomment to perform illumination correction
            #blue = illum_corr(blue)
            green = hist_adj(green, clahe=False, offset=0)
            red = hist_adj(red, clahe=False, offset=0)
                
            mergedImg = cv2.merge([blue, green, red])
            fName = 'merged_' + str(x) + '.TIF'
            print(f"Processed image saved as {fName}", '\n')
            cv2.imwrite(f"{dst_folder}"+fName, mergedImg)
        else:
            print("Image is actually blurry; removed.", '\n')    
    else:
        print("Image is blurry; removed.", '\n')
print("All done!")


Processing image set 1 out of 300. ID=191023110002_C01f100
blue: MFGTMP_191023110002_C01f100d0.TIF
green: MFGTMP_191023110002_C01f100d1.TIF
red: MFGTMP_191023110002_C01f100d2.TIF
Image might be sharp... 1st focus check FM1=1708.20
Image is sharp! 2nd focus check FM2=386.71
Processed image saved as merged_191023110002_C01f100.TIF 


Processing image set 2 out of 300. ID=191023110002_C01f101
blue: MFGTMP_191023110002_C01f101d0.TIF
green: MFGTMP_191023110002_C01f101d1.TIF
red: MFGTMP_191023110002_C01f101d2.TIF
Image might be sharp... 1st focus check FM1=1123.79
Image is sharp! 2nd focus check FM2=464.10
Processed image saved as merged_191023110002_C01f101.TIF 


Processing image set 3 out of 300. ID=191023110002_C01f102
blue: MFGTMP_191023110002_C01f102d0.TIF
green: MFGTMP_191023110002_C01f102d1.TIF
red: MFGTMP_191023110002_C01f102d2.TIF
Image might be sharp... 1st focus check FM1=516.32
Image is sharp! 2nd focus check FM2=286.13
Processed image saved as merged_191023110002_C01f102.TIF 


green: MFGTMP_191023110002_C01f134d1.TIF
red: MFGTMP_191023110002_C01f134d2.TIF
Image is blurry; removed. 


Processing image set 36 out of 300. ID=191023110002_C01f135
blue: MFGTMP_191023110002_C01f135d0.TIF
green: MFGTMP_191023110002_C01f135d1.TIF
red: MFGTMP_191023110002_C01f135d2.TIF
Image is blurry; removed. 


Processing image set 37 out of 300. ID=191023110002_C01f136
blue: MFGTMP_191023110002_C01f136d0.TIF
green: MFGTMP_191023110002_C01f136d1.TIF
red: MFGTMP_191023110002_C01f136d2.TIF
Image might be sharp... 1st focus check FM1=700.62
Image is sharp! 2nd focus check FM2=297.04
Processed image saved as merged_191023110002_C01f136.TIF 


Processing image set 38 out of 300. ID=191023110002_C01f137
blue: MFGTMP_191023110002_C01f137d0.TIF
green: MFGTMP_191023110002_C01f137d1.TIF
red: MFGTMP_191023110002_C01f137d2.TIF
Image might be sharp... 1st focus check FM1=1486.20
Image is sharp! 2nd focus check FM2=380.51
Processed image saved as merged_191023110002_C01f137.TIF 


Processing im

green: MFGTMP_191023110002_C01f160d1.TIF
red: MFGTMP_191023110002_C01f160d2.TIF
Image is blurry; removed. 


Processing image set 63 out of 300. ID=191023110002_C01f161
blue: MFGTMP_191023110002_C01f161d0.TIF
green: MFGTMP_191023110002_C01f161d1.TIF
red: MFGTMP_191023110002_C01f161d2.TIF
Image is blurry; removed. 


Processing image set 64 out of 300. ID=191023110002_C01f162
blue: MFGTMP_191023110002_C01f162d0.TIF
green: MFGTMP_191023110002_C01f162d1.TIF
red: MFGTMP_191023110002_C01f162d2.TIF
Image is blurry; removed. 


Processing image set 65 out of 300. ID=191023110002_C01f163
blue: MFGTMP_191023110002_C01f163d0.TIF
green: MFGTMP_191023110002_C01f163d1.TIF
red: MFGTMP_191023110002_C01f163d2.TIF
Image is blurry; removed. 


Processing image set 66 out of 300. ID=191023110002_C01f164
blue: MFGTMP_191023110002_C01f164d0.TIF
green: MFGTMP_191023110002_C01f164d1.TIF
red: MFGTMP_191023110002_C01f164d2.TIF
Image is blurry; removed. 


Processing image set 67 out of 300. ID=191023110002_C01

green: MFGTMP_191023110002_C01f185d1.TIF
red: MFGTMP_191023110002_C01f185d2.TIF
Image might be sharp... 1st focus check FM1=982.71
Image is sharp! 2nd focus check FM2=264.67
Processed image saved as merged_191023110002_C01f185.TIF 


Processing image set 90 out of 300. ID=191023110002_C01f186
blue: MFGTMP_191023110002_C01f186d0.TIF
green: MFGTMP_191023110002_C01f186d1.TIF
red: MFGTMP_191023110002_C01f186d2.TIF
Image might be sharp... 1st focus check FM1=557.87
Image is actually blurry; removed. 


Processing image set 91 out of 300. ID=191023110002_C01f187
blue: MFGTMP_191023110002_C01f187d0.TIF
green: MFGTMP_191023110002_C01f187d1.TIF
red: MFGTMP_191023110002_C01f187d2.TIF
Image might be sharp... 1st focus check FM1=1553.05
Image is actually blurry; removed. 


Processing image set 92 out of 300. ID=191023110002_C01f188
blue: MFGTMP_191023110002_C01f188d0.TIF
green: MFGTMP_191023110002_C01f188d1.TIF
red: MFGTMP_191023110002_C01f188d2.TIF
Image is blurry; removed. 


Processing image s

red: MFGTMP_191023110002_C01f206d2.TIF
Image is blurry; removed. 


Processing image set 113 out of 300. ID=191023110002_C01f207
blue: MFGTMP_191023110002_C01f207d0.TIF
green: MFGTMP_191023110002_C01f207d1.TIF
red: MFGTMP_191023110002_C01f207d2.TIF
Image is blurry; removed. 


Processing image set 114 out of 300. ID=191023110002_C01f208
blue: MFGTMP_191023110002_C01f208d0.TIF
green: MFGTMP_191023110002_C01f208d1.TIF
red: MFGTMP_191023110002_C01f208d2.TIF
Image is blurry; removed. 


Processing image set 115 out of 300. ID=191023110002_C01f209
blue: MFGTMP_191023110002_C01f209d0.TIF
green: MFGTMP_191023110002_C01f209d1.TIF
red: MFGTMP_191023110002_C01f209d2.TIF
Image is blurry; removed. 


Processing image set 116 out of 300. ID=191023110002_C01f20
blue: MFGTMP_191023110002_C01f200d0.TIF
green: MFGTMP_191023110002_C01f200d1.TIF
red: MFGTMP_191023110002_C01f200d2.TIF
blue: MFGTMP_191023110002_C01f201d0.TIF
green: MFGTMP_191023110002_C01f201d1.TIF
red: MFGTMP_191023110002_C01f201d2.TIF
bl

Image is actually blurry; removed. 


Processing image set 139 out of 300. ID=191023110002_C01f38
blue: MFGTMP_191023110002_C01f38d0.TIF
green: MFGTMP_191023110002_C01f38d1.TIF
red: MFGTMP_191023110002_C01f38d2.TIF
Image might be sharp... 1st focus check FM1=1164.85
Image is actually blurry; removed. 


Processing image set 140 out of 300. ID=191023110002_C01f39
blue: MFGTMP_191023110002_C01f39d0.TIF
green: MFGTMP_191023110002_C01f39d1.TIF
red: MFGTMP_191023110002_C01f39d2.TIF
Image is blurry; removed. 


Processing image set 141 out of 300. ID=191023110002_C01f40
blue: MFGTMP_191023110002_C01f40d0.TIF
green: MFGTMP_191023110002_C01f40d1.TIF
red: MFGTMP_191023110002_C01f40d2.TIF
Image is blurry; removed. 


Processing image set 142 out of 300. ID=191023110002_C01f41
blue: MFGTMP_191023110002_C01f41d0.TIF
green: MFGTMP_191023110002_C01f41d1.TIF
red: MFGTMP_191023110002_C01f41d2.TIF
Image is blurry; removed. 


Processing image set 143 out of 300. ID=191023110002_C01f42
blue: MFGTMP_1910

Image is sharp! 2nd focus check FM2=695.16
Processed image saved as merged_191023110002_C01f74.TIF 


Processing image set 176 out of 300. ID=191023110002_C01f75
blue: MFGTMP_191023110002_C01f75d0.TIF
green: MFGTMP_191023110002_C01f75d1.TIF
red: MFGTMP_191023110002_C01f75d2.TIF
Image is blurry; removed. 


Processing image set 177 out of 300. ID=191023110002_C01f76
blue: MFGTMP_191023110002_C01f76d0.TIF
green: MFGTMP_191023110002_C01f76d1.TIF
red: MFGTMP_191023110002_C01f76d2.TIF
Image is blurry; removed. 


Processing image set 178 out of 300. ID=191023110002_C01f77
blue: MFGTMP_191023110002_C01f77d0.TIF
green: MFGTMP_191023110002_C01f77d1.TIF
red: MFGTMP_191023110002_C01f77d2.TIF
Image is blurry; removed. 


Processing image set 179 out of 300. ID=191023110002_C01f78
blue: MFGTMP_191023110002_C01f78d0.TIF
green: MFGTMP_191023110002_C01f78d1.TIF
red: MFGTMP_191023110002_C01f78d2.TIF
Image is blurry; removed. 


Processing image set 180 out of 300. ID=191023110002_C01f79
blue: MFGTMP_1

red: MFGTMP_191023120002_C01f320d2.TIF
Image might be sharp... 1st focus check FM1=1531.71
Image is actually blurry; removed. 


Processing image set 212 out of 300. ID=191023120002_C01f321
blue: MFGTMP_191023120002_C01f321d0.TIF
green: MFGTMP_191023120002_C01f321d1.TIF
red: MFGTMP_191023120002_C01f321d2.TIF
Image is blurry; removed. 


Processing image set 213 out of 300. ID=191023120002_C01f322
blue: MFGTMP_191023120002_C01f322d0.TIF
green: MFGTMP_191023120002_C01f322d1.TIF
red: MFGTMP_191023120002_C01f322d2.TIF
Image might be sharp... 1st focus check FM1=568.53
Image is sharp! 2nd focus check FM2=430.86
Processed image saved as merged_191023120002_C01f322.TIF 


Processing image set 214 out of 300. ID=191023120002_C01f323
blue: MFGTMP_191023120002_C01f323d0.TIF
green: MFGTMP_191023120002_C01f323d1.TIF
red: MFGTMP_191023120002_C01f323d2.TIF
Image is blurry; removed. 


Processing image set 215 out of 300. ID=191023120002_C01f324
blue: MFGTMP_191023120002_C01f324d0.TIF
green: MFGTMP_1

red: MFGTMP_191023120002_C01f353d2.TIF
Image is blurry; removed. 


Processing image set 245 out of 300. ID=191023120002_C01f354
blue: MFGTMP_191023120002_C01f354d0.TIF
green: MFGTMP_191023120002_C01f354d1.TIF
red: MFGTMP_191023120002_C01f354d2.TIF
Image is blurry; removed. 


Processing image set 246 out of 300. ID=191023120002_C01f355
blue: MFGTMP_191023120002_C01f355d0.TIF
green: MFGTMP_191023120002_C01f355d1.TIF
red: MFGTMP_191023120002_C01f355d2.TIF
Image is blurry; removed. 


Processing image set 247 out of 300. ID=191023120002_C01f356
blue: MFGTMP_191023120002_C01f356d0.TIF
green: MFGTMP_191023120002_C01f356d1.TIF
red: MFGTMP_191023120002_C01f356d2.TIF
Image is blurry; removed. 


Processing image set 248 out of 300. ID=191023120002_C01f357
blue: MFGTMP_191023120002_C01f357d0.TIF
green: MFGTMP_191023120002_C01f357d1.TIF
red: MFGTMP_191023120002_C01f357d2.TIF
Image is blurry; removed. 


Processing image set 249 out of 300. ID=191023120002_C01f358
blue: MFGTMP_191023120002_C01f3

red: MFGTMP_191023120002_C01f389d2.TIF
Image might be sharp... 1st focus check FM1=715.87
Image is actually blurry; removed. 


Processing image set 281 out of 300. ID=191023120002_C01f390
blue: MFGTMP_191023120002_C01f390d0.TIF
green: MFGTMP_191023120002_C01f390d1.TIF
red: MFGTMP_191023120002_C01f390d2.TIF
Image is blurry; removed. 


Processing image set 282 out of 300. ID=191023120002_C01f391
blue: MFGTMP_191023120002_C01f391d0.TIF
green: MFGTMP_191023120002_C01f391d1.TIF
red: MFGTMP_191023120002_C01f391d2.TIF
Image is blurry; removed. 


Processing image set 283 out of 300. ID=191023120002_C01f392
blue: MFGTMP_191023120002_C01f392d0.TIF
green: MFGTMP_191023120002_C01f392d1.TIF
red: MFGTMP_191023120002_C01f392d2.TIF
Image is blurry; removed. 


Processing image set 284 out of 300. ID=191023120002_C01f393
blue: MFGTMP_191023120002_C01f393d0.TIF
green: MFGTMP_191023120002_C01f393d1.TIF
red: MFGTMP_191023120002_C01f393d2.TIF
Image is blurry; removed. 


Processing image set 285 out of 